In [3]:
#Fonte: https://www.vooo.pro/insights/construir-um-modelo-preditivo-em-10-minutos-usando-python/

import pandas as pd
import sys
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [4]:
train=pd.read_csv('C:/Users/Diego/Desktop/pythonRead/train.csv')
test=pd.read_csv('C:/Users/Diego/Desktop/pythonRead/train.csv')
train['Type']='Train' #Cria uma bandeira para o conjunto de dados de treino e de teste
test['Type']='Test'
fullData = pd.concat([train,test],axis=0) #Combina ambos conjuntos de dados de treino e de teste

#Etapa - Verificando os dados

fullData
fullData.columns # This will show all the column names
fullData.head(10) # Show first 10 records of dataframe
fullData.describe() #You can look at summary of numerical fields by using describe() function
fullData


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Type
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Train
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Train
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Train
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Train
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Train
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Train
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Train
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Train
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Train
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Train


In [25]:
#Identifique 
# as (a) variáveis de ID,
#as (b) variáveis-alvo, 
#as (c) as variáveis categóricas,
#as (d) as variáveis numéricas e (e) outras variáveis.

col_id = ['PassengerId']
col_alvo = ['Fare']
col_alvo2 = ['Age'] 
col_categorica = ['Name','Sex']
col_numericas = list(set(list(fullData.columns))-set(col_alvo)-set(col_id)-set(col_alvo2))
outras_colunas = ['Survived']


In [5]:
#Verificando se tem algum valor vazio nas colunas
#Se true é porque tem valor vazio
#Se False é porque não tem

fullData.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
Type           False
dtype: bool

In [32]:
#Combinando Variáveis numéricas com categóricas


num_cat = col_categorica + col_numericas

#Cria uma uma variável para cada variável que tenha valores faltantes com VariableName_NA 
# e marca o valor faltante com 1 e os outros com 0


for var in num_cat:
    if fullData[var].isnull().any() == True:
        fullData[var + '_NA'] = fullData[var].isnull()*1

#Preenche valores faltantes com a média        

fullData[col_numericas] = fullData[col_numericas].fillna(fullData[col_numericas].mean(),inplace = True)
        
#Preenche valores categóricos faltantes com -9999

fullData[col_categorica] = fullData[col_categorica].fillna(value = -9999)



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [39]:
#Crie codificadores de etiqueta para as variáveis categóricas e 
#dividaa o conjunto de dados em treino e teste, divida ainda mais o conjunto de treino 
#em treinamento e validação.


#creia rótulos codificadores para itens categóricos

for var in col_categorica:
    number = LabelEncoder()
    fullData[var] = number.fit_transform(fullData[var].astype('str'))

#Variável alvo também é categórica então é convertida

fullData["Fare"] = number.fit_transform(fullData["Fare"]).astype('str')
fullData["Age"] = number.fit_transform(fullData["Age"]).astype('str')

treinar = fullData[fullData['Type'] == 'Treinar']
test = fullData[fullData['Type'] =='Test']

treinar['is_train'] = np.random.uniform(0,1,len(train)) <=.75
Treinar,Validate = treinar[treinar['is_train'] == True ], treinar[treinar['is_train'] == False]




C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
#Passe para o processo de modelagem as variáveis imputadas e as dummies (valores faltantes marcados com bandeiras). 
#Estou usando Random Forest para prever a classe.

features = list(set(list(fullData.columns))-set(col_id)-set(col_alvo)-set(outras_colunas))



In [41]:
x_treino = Treinar[list(features)].values
y_treino = Treinar["Fare"].values
y_treino2 = Treinar["Age"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Fare"].values
y_validate2 = Validate["Age"].values
x_test = test[list(features)].values


In [47]:
random.seed(100)
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(y_treino ,y_treino2)

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 

In [48]:
#Verifique o desempenho e faça previsões

status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1])
roc_auc = auc(fpr, tpr)
print(roc_auc)

final_status = rf.predict_proba(x_test)
test["Fare"]=final_status[:,1]
test["Age"]=final_status[:,1]

test.to_csv('C:/Users/Diego/Desktop/preditiva.csv',columns=['REF_NO','Account.Status'])


NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.